In [1]:
import pandas as pd

from sentiment_modules import sentiment

2023-12-21 15:59:23.743077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<p style="background-color:#0d3663; font-family: arial; color: #ffffff; font-size: 200%; text-align: center; border-radius: 15px 15px;">Sentiment Analysis</p>

In [2]:
sen = sentiment("restaurant-reviews.csv", "title", "text", "rating")

TextBlob serves, with its lexical approach, as the starting point for sentiment analysis experiments. Quantification relies, therefore, on a pre-defined dictionary of word-sentiment associations, with reference to semantic orientation (positive or negative) and intensity (closer to zero, the weaker). Hence, scores ranging from minus one to one are assigned to each word in given input via dictionary lookups where covered. Results are then averaged, yielding the final sentiment. Prior to that, preprocessing may be beneficial.

Elevating matching accuracy through lemmatization during preprocessing could mitigate out-of-vocabulary occurrences. Even so, lemmatization remains unexplored due to the inherent trade-off at the expense of semantic loss and, consequently, intensity, as gradations are obscured. While standardization for the predominant ASCII character encoding, is deliberately eschewed to safeguard recognized smileys. This choice is rooted in the observation that Unicode equivalents within features generally exhibit comparable semantic orientation and intensity. Noisy escape sequences, for another, are parsed by the library. Other types of noise, whether unrecognized by the library or deemed neutral (such as contractions of stopwords), remain unnoticed.

Comparable impacts of misclassifications among classes advocate accuracy as an evaluation metric. Imbalanced classes, however, challenge this assumption, a situation known as the accuracy paradox. Gauging the model's generalizability across imbalanced classes, accuracy is augmented by its balanced counterpart. Experiment's predictions are therefore evaluated according to both accuracy and balanced accuracy.

Predictions constitute a composite of title and text sentiments. Seperately considered as titles often strive to captivate reader's attention via semtimentally resonant and topically relevant words. It follows that a title tends to exhibit greater discriminatory power than a single sentence within its text. As tuning does not comprise a focal point of this experiment, a generic default weight of 0.5 is applied. A weight of 0.5, as no exploration has yet been conducted, possibly favouring one over the other.

In [3]:
txtblb_sens, txtblb_sens_score = sen.txtblb_sen("txtblb")

txtblb_sens_texts: 100%|██████████| 1000/1000 [00:00<00:00, 1653.88texts/s]


Exploring more sophisticated approaches, transformer models are under consideration, focusing on candidates suitable to NLP, given the textual nature of both text and title. Prediction requirements presuppose text classification related to sentiments. Given that classifications largely hinge on English text data, other language manifestations are not considered.

After review, __["bert-base-multilingual-uncased-sentiment"](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment)__ as fine-tuned by "nlptown" was selected given the following considerations:

- detailed documentation, ensuring smooth onboarding
- fine-tuned based on (product) reviews, in line with the project's data topic
- multilingual, tolerant of linguistic outliers
- predictions mapped to the desired scale
- promising accuracy scores indicative of quality improvements

Preprocessing might be beneficial prior to model application. __[Research](https://towardsdatascience.com/does-bert-need-clean-data-part-2-classification-d29adf9f745a)__ advocates a lighter preprocessing approach, as BERT heavily relies on contextual information. As such, stop words or punctuation contributing to sentence structure are retained, preserving semantic nuance. As part of preprocessing, emoticons were stripped, as they were recognized for their constituents found to cause classification distortions. Another phenomenon observed was similarity between polarity among emojis, consistent in vice versa albeit to a lesser extent to cross-encodings and contractions. Such distortions are mitigated by standardization, enforcing prevalent ASCII character encoding, along with full forms.

In [4]:
bert_gen_sens, bert_gen_sens_score = sen.bert_sen("bert_gen", "nlptown/bert-base-multilingual-uncased-sentiment")

bert_sens_texts: 100%|██████████| 1000/1000 [07:29<00:00,  2.22texts/s]


As the transformer is based on product reviews, diverging from the restaurant subject, fine-tuning may raise quality. Hyperparameters pertaining to fine-tuning were informed by past experiences, constituting well-founded starting values. Examples for fine-tuning comprise concatenated title text strings, reducing computational complexity, as forward passes are halved, in consideration of resource limitations. In the same vein, tuning is deferred. Model evaluation relies on balanced accuracy considering comparable misclassification impacts previously mentioned. 

Evaluation of a model should not be conducted with the same data as training. By evaluating based on training data, data is more likely to be memorized rather than capturing underlying patterns and relationships. Memorization comes at the expense of generalizability to unseen data, leading to the phenomenon of overfitting. Therefore, a split between train and test data is desirable to obtain a realistic preliminary view of the actual classification problem. As part of the hyperparameters, shuffle was enabled to mitigate potential order bias, which may be introduced by the original order of instances. As a means of ensuring that the diversity of the population is adequately reflected within the split, stratification is applied to prevent undercoverage bias. The split of 20% test data and 80% training data was chosen, as 1,000 instances are relatively few in number, necessitating a greater proportion of training data to be able to capture patterns and relationships, with sufficient test size to ensure a reliable quality of evaluation.

The resulting transformer model, __["bert-base-multilingual-uncased-finetuned-sentiment"](https://huggingface.co/thore-dahl/bert-base-multilingual-uncased-finetuned-sentiment)__, is then employed as a further attempt to improve quality.

In [5]:
bert_fitu_sens, bert_fitu_sens_score = sen.bert_sen("bert_fitu", "thore-dahl/bert-base-multilingual-uncased-finetuned-sentiment")

bert_sens_texts: 100%|██████████| 1000/1000 [05:58<00:00,  2.79texts/s]


In [6]:
table = pd.DataFrame(data = [txtblb_sens_score, bert_gen_sens_score, bert_fitu_sens_score])
display(table)

,model,accuracy,balanced_accuracy
0,txtblb,0.337,0.316989
1,bert_gen,0.623,0.542411
2,bert_fitu,0.796,0.733701


Quality unfolded as anticipated, starting with TextBlob, followed by the transformer model, culminating in its fine-tuned equivalent. Leaps waned with progression, yet remained significant. As accuracy exceeds its balanced counterpart, classifications of less prevalent classes tend to suffer. 

Given these findings, production choice gravitates towards the fine-tuned transformer, excelling in both metrics. Observed metric discrepancies in that model may stem from a shortage of representative examples, prompting consideration of data augmentation strategies. The transformer may also be fine-tuned further, configuring hyperparameters for cost-sensitive learning, given access to sufficient computing resources. Moreover, title and text examples may be forward passed separately, aligning training with downstream independent predictions, contributing to the learning of task-specific patterns. Weightings among them could be explored to raise quality even further.